<a href="https://colab.research.google.com/github/Noob919/Ordered-Weighted-Aggregation/blob/main/Ordered-Weighted-Aggregation/Cancer/Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.feature_selection  import chi2, SelectKBest,mutual_info_classif,SelectPercentile,f_classif
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder,StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models,Sequential
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score,precision_score

In [60]:
df = pd.read_csv("https://raw.githubusercontent.com/Noob919/Feature_selection/main/Data/cancer.csv")
df_new = df.drop(['id','Unnamed: 32'],axis =1)
df_new['diagnosis'] = LabelEncoder().fit_transform(df_new['diagnosis'])

In [61]:
X = df_new.drop(['diagnosis'],axis =1)
y = df_new['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

1. Chi Square 

In [4]:
test = SelectKBest(score_func=chi2, k=len(X.columns)).fit(X,y)
np.set_printoptions(precision=3)
score  =  test.scores_
score_df = {'Chi2 ranking': X.columns[score.argsort()[::-1]], 'score': np.sort(score)[::-1]}
ranking_chi = pd.DataFrame(data =  score_df)
ranking_chi

,Chi2 ranking,score
0,area_worst,112598.431564
1,area_mean,53991.655924
2,area_se,8758.504705
3,perimeter_worst,3665.035416
4,perimeter_mean,2011.102864
5,radius_worst,491.689157
6,radius_mean,266.104917
7,perimeter_se,250.571896
8,texture_worst,174.449400
9,texture_mean,93.897508


# Top 50% features

In [ ]:
!pip uninstall tensorflow

In [ ]:
!pip install tensorflow==2.5

# Split the Data and Normalize the data

In [10]:
chi_50_df = df_new[ranking_chi['Chi2 ranking'][0:15]]
X  = chi_50_df
y = df_new['diagnosis']
Scaler = StandardScaler()
X_min =  Scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_min, y, test_size=0.2, random_state=0)
print("Shape of training data",X_train.shape)
print("Shape of testing data",X_test.shape)

Shape of training data (455, 15)
Shape of testing data (114, 15)


Random Forest

In [11]:
rf =  RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred_rf =  rf.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_rf))
print("Recall: ", recall_score(y_test,y_pred_rf))
print("Precision: ", precision_score(y_test,y_pred_rf))
print(confusion_matrix(y_test,y_pred_rf))

Accuracy:  0.9649122807017544
Recall:  0.9574468085106383
Precision:  0.9574468085106383
[[65  2]
 [ 2 45]]


SVM

In [19]:
svc =  SVC()
svc.fit(X_test,y_test)
y_pred_svc =  svc.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_svc))
print("Recall: ", recall_score(y_test,y_pred_svc))
print("Precision: ", precision_score(y_test,y_pred_svc))
print(confusion_matrix(y_test,y_pred_svc))

Accuracy:  0.9649122807017544
Recall:  0.9574468085106383
Precision:  0.9574468085106383
[[65  2]
 [ 2 45]]


ANN

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(15, activation='relu', input_shape=(15,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(28, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])
model.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                240       
_________________________________________________________________
batch_normalization (BatchNo (None, 15)                60        
_________________________________________________________________
dense_1 (Dense)              (None, 20)                320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 20)                80        
_________________________________________________________________
dense_2 (Dense)              (None, 24)                504       
_________________________________________________________________
dropout (Dropout)            (None, 24)                0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 24)                9

In [16]:
h = model.fit(X_train,y_train,epochs=500,validation_data=(X_test,y_test))

Epoch 1/500
15/15 [==============================] - 2s 26ms/step - loss: 0.7687 - accuracy: 0.5341 - val_loss: 0.6325 - val_accuracy: 0.8772
Epoch 2/500
15/15 [==============================] - 0s 4ms/step - loss: 0.6600 - accuracy: 0.6330 - val_loss: 0.6015 - val_accuracy: 0.8947
Epoch 3/500
15/15 [==============================] - 0s 6ms/step - loss: 0.6094 - accuracy: 0.7011 - val_loss: 0.5705 - val_accuracy: 0.8596
Epoch 4/500
15/15 [==============================] - 0s 5ms/step - loss: 0.5543 - accuracy: 0.7253 - val_loss: 0.5393 - val_accuracy: 0.8333
Epoch 5/500
15/15 [==============================] - 0s 4ms/step - loss: 0.4697 - accuracy: 0.7560 - val_loss: 0.5108 - val_accuracy: 0.8333
Epoch 6/500
15/15 [==============================] - 0s 4ms/step - loss: 0.5041 - accuracy: 0.7297 - val_loss: 0.4768 - val_accuracy: 0.8421
Epoch 7/500
15/15 [==============================] - 0s 4ms/step - loss: 0.4786 - accuracy: 0.7868 - val_loss: 0.4455 - val_accuracy: 0.8596
Epoch 8/500


In [20]:
y_pred_ann = model.predict_classes(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_ann))
print("Recall: ", recall_score(y_test,y_pred_ann))
print("Precision: ", precision_score(y_test,y_pred_ann))
print(confusion_matrix(y_test,y_pred_ann))

Accuracy:  0.9385964912280702
Recall:  0.9148936170212766
Precision:  0.9347826086956522
[[64  3]
 [ 4 43]]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


## Top 75% of features

# **Split the data and Normalize **

In [28]:
chi_75_df = df_new[ranking_chi['Chi2 ranking'][0:22]]
X  = chi_75_df
X.head()
y = df_new['diagnosis']
X_min =  Scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_min, y, test_size=0.2, random_state=0)
print("Shape of training data",X_train.shape)
print("Shape of testing data",X_test.shape)

Shape of training data (455, 22)
Shape of testing data (114, 22)


Random Forest 

In [29]:
rf =  RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred_rf =  rf.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_rf))
print("Recall: ", recall_score(y_test,y_pred_rf))
print("Precision: ", precision_score(y_test,y_pred_rf))
print(confusion_matrix(y_test,y_pred_rf))

Accuracy:  0.9649122807017544
Recall:  0.9574468085106383
Precision:  0.9574468085106383
[[65  2]
 [ 2 45]]


SVM

In [30]:
svc =  SVC()
svc.fit(X_test,y_test)
y_pred_svc =  svc.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_svc))
print("Recall: ", recall_score(y_test,y_pred_svc))
print("Precision: ", precision_score(y_test,y_pred_svc))
print(confusion_matrix(y_test,y_pred_svc))

Accuracy:  0.9824561403508771
Recall:  0.9574468085106383
Precision:  1.0
[[67  0]
 [ 2 45]]


ANN

In [35]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(22, activation='relu', input_shape=(22,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(28, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])
model.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 22)                506       
_________________________________________________________________
batch_normalization_12 (Batc (None, 22)                88        
_________________________________________________________________
dense_21 (Dense)             (None, 20)                460       
_________________________________________________________________
batch_normalization_13 (Batc (None, 20)                80        
_________________________________________________________________
dense_22 (Dense)             (None, 24)                504       
_________________________________________________________________
dropout_8 (Dropout)          (None, 24)                0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 24)               

In [36]:
h = model.fit(X_train,y_train,epochs=500,validation_data=(X_test,y_test))

Epoch 1/500
15/15 [==============================] - 1s 14ms/step - loss: 0.7885 - accuracy: 0.4879 - val_loss: 0.6647 - val_accuracy: 0.7105
Epoch 2/500
15/15 [==============================] - 0s 3ms/step - loss: 0.6688 - accuracy: 0.6242 - val_loss: 0.6380 - val_accuracy: 0.7895
Epoch 3/500
15/15 [==============================] - 0s 3ms/step - loss: 0.5664 - accuracy: 0.6901 - val_loss: 0.6067 - val_accuracy: 0.7895
Epoch 4/500
15/15 [==============================] - 0s 3ms/step - loss: 0.5374 - accuracy: 0.7231 - val_loss: 0.5689 - val_accuracy: 0.8070
Epoch 5/500
15/15 [==============================] - 0s 3ms/step - loss: 0.4808 - accuracy: 0.7736 - val_loss: 0.5291 - val_accuracy: 0.8421
Epoch 6/500
15/15 [==============================] - 0s 5ms/step - loss: 0.4256 - accuracy: 0.8110 - val_loss: 0.4882 - val_accuracy: 0.8509
Epoch 7/500
15/15 [==============================] - 0s 3ms/step - loss: 0.4184 - accuracy: 0.8352 - val_loss: 0.4410 - val_accuracy: 0.8596
Epoch 8/500


In [ ]:
y_pred_ann = model.predict_classes(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_ann))
print("Recall: ", recall_score(y_test,y_pred_ann))
print("Precision: ", precision_score(y_test,y_pred_ann))

In [37]:
y_pred_ann = model.predict_classes(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_ann))
print("Recall: ", recall_score(y_test,y_pred_ann))
print("Precision: ", precision_score(y_test,y_pred_ann))
print(confusion_matrix(y_test,y_pred_ann))

Accuracy:  0.956140350877193
Recall:  0.9574468085106383
Precision:  0.9375
[[64  3]
 [ 2 45]]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


# **2. Random Forest feature Importance**

In [39]:
rf = RandomForestClassifier(n_estimators=150)
rf.fit(X, y)
sort = rf.feature_importances_
score_df = {'Tree based method Attributes': X.columns[rf.feature_importances_.argsort()[::-1]],'score': np.sort(rf.feature_importances_)[::-1] }
rank_tree = pd.DataFrame(data =  score_df) 
rank_tree

,Tree based method Attributes,score
0,area_worst,0.152099
1,radius_worst,0.148115
2,perimeter_worst,0.143852
3,concave points_worst,0.103043
4,concave points_mean,0.102913
5,area_mean,0.051538
6,concavity_mean,0.042586
7,area_se,0.035092
8,radius_mean,0.034337
9,concavity_worst,0.032800


# TOP 50%

# **Split the data and Normalise**

In [42]:
tree_50_df = df_new[rank_tree['Tree based method Attributes'][0:15]]
X  = tree_50_df
X.head()
y = df_new['diagnosis']
X_min =  Scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_min, y, test_size=0.2, random_state=0)
print("Shape of training data",X_train.shape)
print("Shape of testing data",X_test.shape)


Shape of training data (455, 15)
Shape of testing data (114, 15)


Random Forest 

In [43]:
rf =  RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred_rf =  rf.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_rf))
print("Recall: ", recall_score(y_test,y_pred_rf))
print("Precision: ", precision_score(y_test,y_pred_rf))
print(confusion_matrix(y_test,y_pred_rf))

Accuracy:  0.9824561403508771
Recall:  0.9787234042553191
Precision:  0.9787234042553191
[[66  1]
 [ 1 46]]


SVM

In [44]:
svc.fit(X_test,y_test)
y_pred_svc =  svc.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_svc))
print("Recall: ", recall_score(y_test,y_pred_svc))
print("Precision: ", precision_score(y_test,y_pred_svc))
print(confusion_matrix(y_test,y_pred_svc))

Accuracy:  0.9736842105263158
Recall:  0.9361702127659575
Precision:  1.0
[[67  0]
 [ 3 44]]


ANN

In [45]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(15, activation='relu', input_shape=(15,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(28, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])
model.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 15)                240       
_________________________________________________________________
batch_normalization_15 (Batc (None, 15)                60        
_________________________________________________________________
dense_26 (Dense)             (None, 20)                320       
_________________________________________________________________
batch_normalization_16 (Batc (None, 20)                80        
_________________________________________________________________
dense_27 (Dense)             (None, 24)                504       
_________________________________________________________________
dropout_10 (Dropout)         (None, 24)                0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 24)               

In [46]:
h = model.fit(X_train,y_train,epochs=500,validation_data=(X_test,y_test))

Epoch 1/500
15/15 [==============================] - 4s 30ms/step - loss: 0.8309 - accuracy: 0.5253 - val_loss: 0.6551 - val_accuracy: 0.7544
Epoch 2/500
15/15 [==============================] - 0s 5ms/step - loss: 0.6945 - accuracy: 0.5934 - val_loss: 0.6045 - val_accuracy: 0.8509
Epoch 3/500
15/15 [==============================] - 0s 5ms/step - loss: 0.6189 - accuracy: 0.6813 - val_loss: 0.5550 - val_accuracy: 0.9474
Epoch 4/500
15/15 [==============================] - 0s 6ms/step - loss: 0.5647 - accuracy: 0.7275 - val_loss: 0.5033 - val_accuracy: 0.9386
Epoch 5/500
15/15 [==============================] - 0s 7ms/step - loss: 0.4990 - accuracy: 0.7714 - val_loss: 0.4513 - val_accuracy: 0.9298
Epoch 6/500
15/15 [==============================] - 0s 5ms/step - loss: 0.4490 - accuracy: 0.8000 - val_loss: 0.3995 - val_accuracy: 0.9298
Epoch 7/500
15/15 [==============================] - 0s 6ms/step - loss: 0.4657 - accuracy: 0.8044 - val_loss: 0.3557 - val_accuracy: 0.9474
Epoch 8/500


In [47]:
y_pred_ann = model.predict_classes(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_ann))
print("Recall: ", recall_score(y_test,y_pred_ann))
print("Precision: ", precision_score(y_test,y_pred_ann))
print(confusion_matrix(y_test,y_pred_ann))

Accuracy:  0.9736842105263158
Recall:  0.9787234042553191
Precision:  0.9583333333333334
[[65  2]
 [ 1 46]]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


# **TOP 75%**

# **Spilt the data and Normalise**

In [50]:
tree_75_df = df_new[rank_tree['Tree based method Attributes'][0:22]]
X  = tree_75_df
X.head()
y = df_new['diagnosis']
X_min =  Scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_min, y, test_size=0.2, random_state=0)
print("Shape of training data",X_train.shape)
print("Shape of testing data",X_test.shape)

Shape of training data (455, 22)
Shape of testing data (114, 22)


Random Forest 

In [51]:
rf =  RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred_rf =  rf.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_rf))
print("Recall: ", recall_score(y_test,y_pred_rf))
print("Precision: ", precision_score(y_test,y_pred_rf))
print(confusion_matrix(y_test,y_pred_rf))

Accuracy:  0.9649122807017544
Recall:  0.9574468085106383
Precision:  0.9574468085106383
[[65  2]
 [ 2 45]]


SVM

In [53]:
svc =  SVC()
svc.fit(X_test,y_test)
y_pred_svc =  svc.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_svc))
print("Recall: ", recall_score(y_test,y_pred_svc))
print("Precision: ", precision_score(y_test,y_pred_svc))
print(confusion_matrix(y_test,y_pred_svc))

Accuracy:  0.9824561403508771
Recall:  0.9574468085106383
Precision:  1.0
[[67  0]
 [ 2 45]]


ANN

In [54]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(22, activation='relu', input_shape=(22,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(28, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])
model.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 22)                506       
_________________________________________________________________
batch_normalization_18 (Batc (None, 22)                88        
_________________________________________________________________
dense_31 (Dense)             (None, 20)                460       
_________________________________________________________________
batch_normalization_19 (Batc (None, 20)                80        
_________________________________________________________________
dense_32 (Dense)             (None, 24)                504       
_________________________________________________________________
dropout_12 (Dropout)         (None, 24)                0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 24)               

In [55]:
h = model.fit(X_train,y_train,epochs=500,validation_data=(X_test,y_test))

Epoch 1/500
15/15 [==============================] - 2s 21ms/step - loss: 0.7579 - accuracy: 0.5934 - val_loss: 0.6337 - val_accuracy: 0.8772
Epoch 2/500
15/15 [==============================] - 0s 5ms/step - loss: 0.6396 - accuracy: 0.6923 - val_loss: 0.5843 - val_accuracy: 0.9123
Epoch 3/500
15/15 [==============================] - 0s 5ms/step - loss: 0.6113 - accuracy: 0.6989 - val_loss: 0.5321 - val_accuracy: 0.9123
Epoch 4/500
15/15 [==============================] - 0s 5ms/step - loss: 0.5129 - accuracy: 0.7538 - val_loss: 0.4794 - val_accuracy: 0.9123
Epoch 5/500
15/15 [==============================] - 0s 4ms/step - loss: 0.4660 - accuracy: 0.7890 - val_loss: 0.4252 - val_accuracy: 0.9123
Epoch 6/500
15/15 [==============================] - 0s 4ms/step - loss: 0.4066 - accuracy: 0.8176 - val_loss: 0.3780 - val_accuracy: 0.9123
Epoch 7/500
15/15 [==============================] - 0s 4ms/step - loss: 0.3664 - accuracy: 0.8593 - val_loss: 0.3244 - val_accuracy: 0.9123
Epoch 8/500


In [56]:
y_pred_ann = model.predict_classes(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_ann))
print("Recall: ", recall_score(y_test,y_pred_ann))
print("Precision: ", precision_score(y_test,y_pred_ann))
print(confusion_matrix(y_test,y_pred_ann))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy:  0.956140350877193
Recall:  1.0
Precision:  0.9038461538461539
[[62  5]
 [ 0 47]]


3. Permuatation Importance mean

In [63]:
clf = LogisticRegression(solver = 'liblinear').fit(X_train, y_train)
result = permutation_importance(clf, X, y, n_repeats=10, random_state=0)
score_df = {'Permutation_importance_mean': X.columns[result.importances_mean.argsort()[::-1]],'score': np.sort(result.importances_mean)[::-1]}
rank_per_mean = pd.DataFrame(data =  score_df) 
rank_per_mean

,Permutation_importance_mean,score
0,area_worst,0.369069
1,perimeter_worst,0.315817
2,area_mean,0.149912
3,area_se,0.100703
4,radius_mean,0.087698
5,radius_worst,0.063445
6,texture_worst,0.055360
7,perimeter_mean,0.024077
8,concavity_worst,0.002109
9,radius_se,0.001582


# TOP 50%

# **Split the data and Normalize**

In [64]:
permutation_mean_50_df = df_new[rank_per_mean['Permutation_importance_mean'][0:15]]
X  = permutation_mean_50_df
X.head()
y = df_new['diagnosis']
X_min =  Scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_min, y, test_size=0.2, random_state=0)
print("Shape of training data",X_train.shape)
print("Shape of testing data",X_test.shape)

Shape of training data (455, 15)
Shape of testing data (114, 15)


Random Forest 

In [65]:
rf =  RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred_rf =  rf.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_rf))
print("Recall: ", recall_score(y_test,y_pred_rf))
print("Precision: ", precision_score(y_test,y_pred_rf))
print(confusion_matrix(y_test,y_pred_rf))

Accuracy:  0.9473684210526315
Recall:  0.9787234042553191
Precision:  0.9019607843137255
[[62  5]
 [ 1 46]]


SVM

In [66]:
svc =  SVC()
svc.fit(X_test,y_test)
y_pred_svc =  svc.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_svc))
print("Recall: ", recall_score(y_test,y_pred_svc))
print("Precision: ", precision_score(y_test,y_pred_svc))
print(confusion_matrix(y_test,y_pred_svc))

Accuracy:  0.9736842105263158
Recall:  0.9787234042553191
Precision:  0.9583333333333334
[[65  2]
 [ 1 46]]


ANN

In [67]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(15, activation='relu', input_shape=(15,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(28, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])
model.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 15)                240       
_________________________________________________________________
batch_normalization_21 (Batc (None, 15)                60        
_________________________________________________________________
dense_36 (Dense)             (None, 20)                320       
_________________________________________________________________
batch_normalization_22 (Batc (None, 20)                80        
_________________________________________________________________
dense_37 (Dense)             (None, 24)                504       
_________________________________________________________________
dropout_14 (Dropout)         (None, 24)                0         
_________________________________________________________________
batch_normalization_23 (Batc (None, 24)               

In [68]:
h = model.fit(X_train,y_train,epochs=500,validation_data=(X_test,y_test))

Epoch 1/500
15/15 [==============================] - 2s 21ms/step - loss: 0.9370 - accuracy: 0.4769 - val_loss: 0.6211 - val_accuracy: 0.6579
Epoch 2/500
15/15 [==============================] - 0s 4ms/step - loss: 0.7407 - accuracy: 0.5714 - val_loss: 0.5758 - val_accuracy: 0.8684
Epoch 3/500
15/15 [==============================] - 0s 4ms/step - loss: 0.6336 - accuracy: 0.6308 - val_loss: 0.5252 - val_accuracy: 0.8860
Epoch 4/500
15/15 [==============================] - 0s 5ms/step - loss: 0.5415 - accuracy: 0.7385 - val_loss: 0.4781 - val_accuracy: 0.8860
Epoch 5/500
15/15 [==============================] - 0s 4ms/step - loss: 0.5176 - accuracy: 0.7626 - val_loss: 0.4341 - val_accuracy: 0.8684
Epoch 6/500
15/15 [==============================] - 0s 4ms/step - loss: 0.4216 - accuracy: 0.8264 - val_loss: 0.3998 - val_accuracy: 0.8684
Epoch 7/500
15/15 [==============================] - 0s 6ms/step - loss: 0.4139 - accuracy: 0.8286 - val_loss: 0.3665 - val_accuracy: 0.8860
Epoch 8/500


In [69]:
y_pred_ann = model.predict_classes(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred_ann))
print("Recall: ", recall_score(y_test,y_pred_ann))
print("Precision: ", precision_score(y_test,y_pred_ann))
print(confusion_matrix(y_test,y_pred_ann))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy:  0.956140350877193
Recall:  1.0
Precision:  0.9038461538461539
[[62  5]
 [ 0 47]]


## TOP 75%

## **Split the data and Normalise**

In [70]:
permutation_mean_75_df = df_new[rank_per_mean['Permutation_importance_mean'][0:22]]
X  = permutation_mean_75_df
X.head()
y = df_new['diagnosis']
X_min =  Scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_min, y, test_size=0.2, random_state=0)
print("Shape of training data",X_train.shape)
print("Shape of testing data",X_test.shape)

Shape of training data (455, 22)
Shape of testing data (114, 22)


4. Permutation Importance Std

In [21]:
score_df = {'Permutation_importance_std': X.columns[result.importances_std.argsort()[::-1]],'score': np.sort(result.importances_std)[::-1]}
rank_per_std = pd.DataFrame(data =  score_df) 
rank_per_std

,Permutation_importance_std,score
0,perimeter_worst,0.017681
1,area_worst,0.016930
2,area_mean,0.015078
3,radius_worst,0.012364
4,area_se,0.012329
5,texture_worst,0.009275
6,radius_mean,0.008408
7,perimeter_mean,0.005036
8,texture_se,0.004236
9,texture_mean,0.003806


5. PCA

In [22]:
pca = PCA().fit(X)
loadings = pd.DataFrame(
    data=pca.components_.T * np.sqrt(pca.explained_variance_), 
    columns=[f'PC{i}' for i in range(1, len(X.columns) + 1)],
    index=X.columns
)
loadings.head()
rank_pca = pd.DataFrame(loadings['PC1'])
rank_pca =  rank_pca.sort_values(by='PC1', ascending = False)
rank_pca

,PC1
area_worst,567.619157
area_mean,344.294342
area_se,37.123772
perimeter_worst,32.947204
perimeter_mean,23.366802
radius_worst,4.766269
radius_mean,3.388296
texture_worst,2.043388
perimeter_se,1.489784
texture_mean,1.463289


7. f score

In [23]:
fs_fit_fscore = SelectKBest(f_classif, k=len(X.columns))
fs_fit_fscore.fit_transform(X, y)
fs_indices_fscore = np.argsort(np.nan_to_num(fs_fit_fscore.scores_))[::-1][0:len(X.columns)]
fs_indices_fscore
best_features_fscore = X.columns[fs_indices_fscore].values
feature_importances_fscore = fs_fit_fscore.scores_[fs_indices_fscore]
score_df = {'Score':np.sort(feature_importances_fscore)[::-1], 'Name':best_features_fscore }
rank_f_test = pd.DataFrame(score_df) 
rank_f_test

,Score,Name
0,964.385393,concave points_worst
1,897.944219,perimeter_worst
2,861.676020,concave points_mean
3,860.781707,radius_worst
4,697.235272,perimeter_mean
5,661.600206,area_worst
6,646.981021,radius_mean
7,573.060747,area_mean
8,533.793126,concavity_mean
9,436.691939,concavity_worst


8. Mutual Info

In [24]:
fs_fit_mutual_info = SelectKBest(mutual_info_classif, k=len(X.columns))
fs_fit_mutual_info.fit_transform(X, y)
fs_indices_mutual_info = np.argsort(fs_fit_mutual_info.scores_)[::-1][0:len(X.columns)]
best_features_mutual_info = X.columns[fs_indices_mutual_info].values
feature_importances_mutual_info = fs_fit_mutual_info.scores_[fs_indices_mutual_info]
score_df = {'Score':np.sort(feature_importances_mutual_info)[::-1], 'Name':best_features_mutual_info }
rank_mutual = pd.DataFrame(score_df)
rank_mutual

,Score,Name
0,0.476442,perimeter_worst
1,0.464704,area_worst
2,0.451967,radius_worst
3,0.440578,concave points_mean
4,0.434678,concave points_worst
5,0.403621,perimeter_mean
6,0.373525,concavity_mean
7,0.365946,radius_mean
8,0.358769,area_mean
9,0.339412,area_se


9. RFE

In [25]:
from sklearn.feature_selection import RFE
from sklearn.svm import LinearSVC

In [28]:
estimator = LinearSVC()
selector = RFE(estimator, n_features_to_select=1, step=1)
selector = selector.fit(X, y)
index = selector.ranking_.argsort()
score_df = {'Rank_RFE': X.columns[index]}
rank_RFE = pd.DataFrame(data = score_df) 
rank_RFE

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

,Rank_RFE
0,concave points_worst
1,concavity_mean
2,concavity_worst
3,radius_worst
4,radius_mean
5,perimeter_worst
6,perimeter_se
7,compactness_worst
8,area_se
9,texture_se
